In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241211_02_SparkSQL").getOrCreate()

24/12/11 15:28:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
trips_df = spark.read.csv('data/fhvhv_tripdata_2020-03.csv', inferSchema=True, header=True)

In [4]:
trips_df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [5]:
trips_df.createOrReplaceTempView('mobility_data')

In [ ]:
query = '''
select *
from mobility_data
limit 5
'''
spark.sql(query).show()

In [9]:
query_1='''
select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data
group by pickup_date
'''
spark.sql(query_1).show()
#전체가 대상인 코드 

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-16|391518|
| 2020-03-03|697880|
| 2020-03-06|872012|
| 2020-03-26|141607|
| 2020-03-05|731165|
| 2020-03-02|648986|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
| 2020-03-04|707879|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-13|660914|
| 2020-03-27|159339|
| 2020-03-22|162165|
| 2020-03-28|138456|
| 2020-03-01|784246|
| 2020-03-19|252773|
| 2020-03-09|628940|
+-----------+------+
only showing top 20 rows



In [8]:
query_2='''
SELECT 
     pickup_date, 
    count(*) as trips
FROM ( SELECT
    split(pickup_datetime, ' ')[0] as pickup_date
    FROM mobility_data )
group by pickup_date
''' #FROM 안에 먼저 처리 위와 아래 차이  

In [10]:
spark.sql(query_2).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-16|391518|
| 2020-03-03|697880|
| 2020-03-06|872012|
| 2020-03-26|141607|
| 2020-03-05|731165|
| 2020-03-02|648986|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
| 2020-03-04|707879|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-13|660914|
| 2020-03-27|159339|
| 2020-03-22|162165|
| 2020-03-28|138456|
| 2020-03-01|784246|
| 2020-03-19|252773|
| 2020-03-09|628940|
+-----------+------+
only showing top 20 rows



In [11]:
spark.sql(query_1).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[split(pickup_datetime#18,  , -1)[0]#106], functions=[count(1)])
+- Exchange hashpartitioning(split(pickup_datetime#18,  , -1)[0]#106, 200), ENSURE_REQUIREMENTS, [id=#100]
   +- *(1) HashAggregate(keys=[split(pickup_datetime#18,  , -1)[0] AS split(pickup_datetime#18,  , -1)[0]#106], functions=[partial_count(1)])
      +- FileScan csv [pickup_datetime#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lab10/git_project/src/data/fhvhv_tripdata_2020-03.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<pickup_datetime:string>




In [12]:
spark.sql(query_2).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[pickup_date#109], functions=[count(1)])
+- Exchange hashpartitioning(pickup_date#109, 200), ENSURE_REQUIREMENTS, [id=#122]
   +- *(1) HashAggregate(keys=[pickup_date#109], functions=[partial_count(1)])
      +- *(1) Project [split(pickup_datetime#18,  , -1)[0] AS pickup_date#109]
         +- FileScan csv [pickup_datetime#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lab10/git_project/src/data/fhvhv_tripdata_2020-03.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<pickup_datetime:string>




In [ ]:
#spark 안에서는 거의 차이가 없다. 

In [13]:
spark.stop()